Use model trained with the notebook below.

https://www.kaggle.com/code/harsha1999/rsna-2-5d-cnn-training-pytorch

In [ ]:
!pip install -q /kaggle/input/rsna-atd-whl-ds/python_gdcm-3.0.22-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -q /kaggle/input/rsna-atd-whl-ds/pylibjpeg-1.4.0-py3-none-any.whl

In [ ]:
import os
import random
import pickle
import gc
from pathlib import Path

import cv2
from glob import glob
import gdcm
import pydicom
import zipfile
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm.notebook import tqdm
from tabulate import tabulate
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam
import torchvision
from torchvision import models
from torchvision.transforms.v2 import Resize, Compose, RandomHorizontalFlip, ColorJitter, RandomAffine, RandomErasing, ToTensor
import torchvision.transforms.v2 as t

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Conv2d(4, 3, kernel_size=3)
        model = models.efficientnet_b0(weights='IMAGENET1K_V1')
        
        self.features = model.features
        self.avgpool = model.avgpool
        
        self.bowel = nn.Linear(1280, 1)
        self.extravasation = nn.Linear(1280, 1)
        self.kidney = nn.Linear(1280, 3)
        self.liver = nn.Linear(1280,3) 
        self.spleen = nn.Linear(1280, 3)
    
    def forward(self, x):
        x = self.input(x)
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        
        bowel = self.bowel(x)
        extravsation = self.extravasation(x)
        kidney = self.kidney(x)
        liver = self.liver(x)
        spleen = self.spleen(x)
        
        return bowel, extravsation, kidney, liver, spleen

In [ ]:
# FIXME: Replace with your model
RNSAModel = CNNModel
model = torch.load("/kaggle/input/rnsa-2023-atd-b0-2433/efficientnet_b0_2.433.pth", map_location=torch.device(device))
model.eval()

In [ ]:
def standardize_pixel_array(dcm: pydicom.dataset.FileDataset) -> np.ndarray:
    """
    https://www.kaggle.com/competitions/rsna-2023-abdominal-trauma-detection/discussion/427217
    """
    pixel_array = dcm.pixel_array
    if dcm.PixelRepresentation == 1:
        bit_shift = dcm.BitsAllocated - dcm.BitsStored
        dtype = pixel_array.dtype 
        pixel_array = (pixel_array << bit_shift).astype(dtype) >>  bit_shift
    intercept = float(dcm.RescaleIntercept)
    slope = float(dcm.RescaleSlope)
    center = int(dcm.WindowCenter)
    width = int(dcm.WindowWidth)
    low = center - width / 2
    high = center + width / 2    
    pixel_array = (pixel_array * slope) + intercept
    pixel_array = np.clip(pixel_array, low, high)
    return pixel_array


def select_elements_with_spacing(input_list, spacing):        
    lower_bound = int(len(input_list) * 0.4)
    upper_bound = int(len(input_list) * 0.6)
    spacing = (upper_bound - lower_bound) // 3
    selected_indices = [
        lower_bound, 
        lower_bound + spacing, 
        lower_bound + (2 * spacing), 
        upper_bound
    ]
    selected_elements = [input_list[index] for index in selected_indices]
    return selected_elements


def standardize_pixel_array(dicom_image):
    pixel_array = dicom_image.pixel_array
    if dicom_image.PixelRepresentation == 1:
        bit_shift = dicom_image.BitsAllocated - dicom_image.BitsStored
        dtype = pixel_array.dtype 
        new_array = (pixel_array << bit_shift).astype(dtype) >>  bit_shift
        pixel_array = pydicom.pixel_data_handlers.util.apply_modality_lut(new_array, dicom_image)
    if dicom_image.PhotometricInterpretation == "MONOCHROME1":
        pixel_array = 1 - pixel_array
    # transform to hounsfield units
    intercept = dicom_image.RescaleIntercept
    slope = dicom_image.RescaleSlope
    pixel_array = pixel_array * slope + intercept
    # windowing
    window_center = int(dicom_image.WindowCenter)
    window_width = int(dicom_image.WindowWidth)
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    pixel_array = pixel_array.copy()
    pixel_array[pixel_array < img_min] = img_min
    pixel_array[pixel_array > img_max] = img_max
    # normalization
    if pixel_array.max() == pixel_array.min():
        pixel_array = np.zeros_like(pixel_array)  # Handle case of constant array
    else:
        pixel_array = (pixel_array - pixel_array.min()) / (pixel_array.max() - pixel_array.min())
    return pixel_array


def preprocess_jpeg(jpeg_path):
    img = cv2.imread(jpeg_path)
    greyscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)/255
    return greyscale


class MetricsCalculator:
    def __init__(self, mode='binary'):
        self.probabilities = []
        self.predictions = []
        self.targets = []
        self.mode = mode
    
    def update(self, logits, target):
        if self.mode == 'binary':
            probabilities = torch.sigmoid(logits)
            predicted = (probabilities > 0.5)
        else:
            probabilities = F.softmax(logits, dim = 1)
            predicted = torch.argmax(probabilities, dim=1)
        self.probabilities.extend(probabilities.detach().cpu().numpy())
        self.predictions.extend(predicted.detach().cpu().numpy())
        self.targets.extend(target.detach().cpu().numpy())
    
    def reset(self):
        self.probabilities = []
        self.predictions = []
        self.targets = []
    
    def compute_accuracy(self):
        return accuracy_score(self.targets, self.predictions)
    
    def compute_auc(self):
        if self.mode == 'multi':
            return roc_auc_score(self.targets, self.probabilities, multi_class = 'ovo', labels=[0, 1, 2])
        else:
            return roc_auc_score(self.targets, self.probabilities)
        
        
def make_main_df(meta_filepath, root_dcm_dirpath, root_image_dirpath):
    meta_df = pd.read_csv(meta_filepath)
    unique_patients = meta_df["patient_id"].nunique()
    meta_df["dicom_folder"] = root_dcm_dirpath + "/" + meta_df.patient_id.astype(str) + "/" + meta_df.series_id.astype(str)
    dcm_dirpaths = meta_df.dicom_folder.tolist()
    dcm_filepaths = []
    for folder in tqdm(dcm_dirpaths):
        _dcm_filepaths = sorted(glob(os.path.join(folder, "*dcm")))
        if len(_dcm_filepaths) == 5:
            _dcm_filepaths = [_dcm_filepaths[0], _dcm_filepaths[1], _dcm_filepaths[3], _dcm_filepaths[4]]
        elif len(_dcm_filepaths) == 6:
            _dcm_filepaths = [_dcm_filepaths[0], _dcm_filepaths[2], _dcm_filepaths[3], _dcm_filepaths[5]]
        elif len(_dcm_filepaths) == 7:
            _dcm_filepaths = [_dcm_filepaths[0], _dcm_filepaths[2], _dcm_filepaths[4], _dcm_filepaths[6]]
        elif len(_dcm_filepaths) >= 8:
            _dcm_filepaths = select_elements_with_spacing(_dcm_filepaths, spacing=2)
        dcm_filepaths += _dcm_filepaths
    main_df = pd.DataFrame(dcm_filepaths, columns=["dicom_filepath"])
    main_df["patient_id"] = main_df.dicom_filepath.map(lambda x: x.split("/")[-3]).astype(int)
    main_df["series_id"] = main_df.dicom_filepath.map(lambda x: x.split("/")[-2]).astype(int)
    main_df["instance_number"] = main_df.dicom_filepath.map(lambda x: x.split("/")[-1].replace(".dcm","")).astype(int)
    main_df["image_filepath"] = f"{root_image_dirpath}" + "/" + main_df.patient_id.astype(str) + "/" + main_df.series_id.astype(str) + "/" + main_df.instance_number.astype(str) + ".png"
    return main_df
    

def process(dicom_filepath, image_filepath, size=512):
    image_dirpath = str(Path(image_filepath).parent)
    os.makedirs(image_dirpath, exist_ok=True)
    dicom = pydicom.dcmread(dicom_filepath)
    pos_z = dicom[(0x20, 0x32)].value[-1]
    image = standardize_pixel_array(dicom)
    image = (image - image.min()) / (image.max() - image.min() + 1e-6)
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        image = 1 - image
    if size is not None:
        image = cv2.resize(image, (size, size))
    cv2.imwrite(image_filepath, (image * 255).astype(np.uint8))

In [ ]:
# make dataframe
root_image_dirpath = "rsna-2023-test-images-256"
meta_filepath = f"/kaggle/input/rsna-2023-abdominal-trauma-detection/test_series_meta.csv"
root_dcm_dirpath = f"/kaggle/input/rsna-2023-abdominal-trauma-detection/test_images"
    
main_df = make_main_df(meta_filepath, root_dcm_dirpath, root_image_dirpath)
main_df.head(4)

In [ ]:
# preprocess images
patients = os.listdir(root_dcm_dirpath)
for idx, row in tqdm(main_df.iterrows()):
    patient_id = row["patient_id"]
    series_id = row["series_id"]
    instance_number = row["instance_number"]
    dicom_filepath = row["dicom_filepath"]
    image_filepath = row["image_filepath"]
    process(dicom_filepath, image_filepath, size=256)

In [ ]:
class AbdominalData(Dataset):
    def __init__(self, df, spacing=2):
        super().__init__()
        self.data = self.fetch(df)
        self.spacing = spacing
        self.transform = Compose([
            ToTensor()
        ])
        
    def fetch(self, df):
        data = []
        print('Scanning df...')
        patient_ids = df['patient_id'].unique()
        for patient_idx, patient_id in tqdm(enumerate(patient_ids)):
            patient_df = df[df['patient_id'] == patient_id]
            series_ids = patient_df['series_id']
            for series_id in series_ids:
                series_df = patient_df[patient_df['series_id'] == series_id]
                image_filepaths = list(series_df['image_filepath'].astype(str))
                data.append({
                    'patient_idx': patient_idx, 
                    'patient_id': patient_id, 
                    'image_filepaths': image_filepaths
                })
        return data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        cur_data = self.data[idx]
        patient_idx, patient_id, image_filepaths = cur_data['patient_idx'], cur_data['patient_id'], cur_data['image_filepaths']
        if len(image_filepaths) == 1:
            dicom_images = [image_filepaths[0]] * 4
        elif len(image_filepaths) == 2:
            dicom_images = [image_filepaths[0], image_filepaths[0], image_filepaths[1], image_filepaths[1]]
        elif len(image_filepaths) == 3:
            dicom_images = [image_filepaths[0], image_filepaths[1], image_filepaths[2], image_filepaths[2]]
        elif len(image_filepaths) == 4:
            dicom_images = [image_filepaths[0], image_filepaths[1], image_filepaths[2], image_filepaths[3]]
        elif len(image_filepaths) >= 5:
            raise NotImplementedError()
        images = [preprocess_jpeg(_) for _ in dicom_images]
        images = np.stack(images)
        image = torch.tensor(images, dtype=torch.float).unsqueeze(dim=1)
        image = self.transform(image).squeeze(dim = 1)
        item = {'image': image, 'patient_idx': patient_idx, 'patient_id': patient_id}
        return item

In [ ]:
test_data = AbdominalData(main_df)
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=False)

In [ ]:
def post_proc(bowel, extravasation, kidney, liver, spleen):
    proc_pred = np.empty((2 * 2 + 3 * 3), dtype="float32")
    # bowel, extravasation
    proc_pred[0] = bowel
    proc_pred[1] = 1 - bowel
    proc_pred[2] = extravasation
    proc_pred[3] = 1 - extravasation
    # kidney, liver, spleen
    proc_pred[4:7] = kidney
    proc_pred[7:10] = liver
    proc_pred[10:13] = spleen
    return proc_pred


patient_ids = main_df["patient_id"].unique()
patient_preds = np.zeros(shape=(len(patient_ids), 2 * 2 + 3 * 3), dtype="float32")
patient_pred_counts = np.zeros(shape=(len(patient_ids), 1), dtype="float32")

for batch_idx, batch_data in enumerate(tqdm(test_dataloader)):                                  
    batch_inputs = batch_data['image'].to(device)
    batch_patient_idxs = batch_data['patient_idx']
    batch_patient_ids = batch_data['patient_id']
    
    bowels, extravsations, kidneys, livers, spleens = model(batch_inputs)
    bowels = torch.sigmoid(bowels).detach().cpu().numpy()
    extravsations = torch.sigmoid(extravsations).detach().cpu().numpy()
    kidneys = torch.softmax(kidneys, dim=1).detach().cpu().numpy()
    livers = torch.softmax(livers, dim=1).detach().cpu().numpy()
    spleens = torch.softmax(spleens, dim=1).detach().cpu().numpy()
    
    for patient_idx, bowel, extravsation, kidney, liver, spleen in zip(batch_data['patient_idx'], bowels, extravsations, kidneys, livers, spleens):
        patient_preds[patient_idx, :] += post_proc(bowel, extravsation, kidney, liver, spleen)
        patient_pred_counts[patient_idx, 0] += 1.0
    
    del bowels, extravsations, kidneys, livers, spleens
    del batch_inputs, batch_patient_ids
    gc.collect()
    
patient_preds = patient_preds / patient_pred_counts

## Submission

In [ ]:
# Create Submission
TARGET_COLS  = [
    "bowel_healthy", "bowel_injury",
    "extravasation_healthy", "extravasation_injury",
    "kidney_healthy", "kidney_low", "kidney_high",
    "liver_healthy", "liver_low", "liver_high",
    "spleen_healthy", "spleen_low", "spleen_high"
]
pred_df = pd.DataFrame({"patient_id":patient_ids,})
pred_df[TARGET_COLS] = patient_preds.astype("float32")

# Align with sample submission
sub_df = pd.read_csv(f"/kaggle/input/rsna-2023-abdominal-trauma-detection/sample_submission.csv")
sub_df = sub_df[["patient_id"]]
sub_df = sub_df.merge(pred_df, on="patient_id", how="left")

# Store submission
sub_df.to_csv("submission.csv",index=False)
sub_df.head(4)